In [2]:
import util
from openai import OpenAI
import re
import os
import prompt_template
import pandas as pd


In [3]:
def call_LLM(user_prompt, system_prompt):
    client = OpenAI(api_key = os.environ["GRAPHRAG_API_KEY"])
    response = client.chat.completions.create(
        model="gpt-4o-2024-08-06", #"gpt-4o-2024-08-06",  gpt-4o-mini-2024-07-18   
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        #max_tokens = 4096,  
        temperature = 0.2
    )

    return response


# Prompt 1

In [14]:
MODEL = "GPT-OP"
folder_path = f'output/search_results/{MODEL}'
output_folder_path = f'output/query_results/{MODEL}/prompt_1'


for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):

        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            req_type = ""
            if "nfr" in filename:
                req_type = "Non-Functional requirement"
            else:
                req_type = "Functional requirement"
            reference_text = file.read()
            with open("documents/requirements/" + filename, 'r') as req_file:
                requirement = req_file.read()
                user_prompt = prompt_template.prompt_query_graph_I(requirement, req_type)
                system_prompt = f"You are a software requirement analyst trained on standards, articles, and best practices contained in the following reference material: {reference_text}."
                result = call_LLM(user_prompt, system_prompt)
                text_result = result.choices[0].message.content
                print(text_result)

                output_file_path = os.path.join(output_folder_path, filename)
                with open(output_file_path, 'w') as output_file:
                    output_file.write(text_result)
                    print(f"Written content to {output_file_path}")

            






SUMMARY:
- Total incompleteness issues found: 3
- Total violations found: 1

DETAILED ANALYSIS:

**Issue Type:** Incompleteness  
**Issue Title:** Lack of Load Testing Criteria  
**Explanation:** The requirement does not specify the need for load testing to validate the performance criteria under expected and peak loads. According to best practices in performance engineering, load testing is essential to ensure that the system can handle the specified number of simultaneous users and requests without degradation.  
**Suggested Solution:** Include a requirement for comprehensive load testing to be conducted prior to deployment, simulating 4 million simultaneous users and requests to validate performance metrics.

---

**Issue Type:** Incompleteness  
**Issue Title:** Absence of Monitoring and Alerting Mechanisms  
**Explanation:** The requirement lacks details on monitoring and alerting mechanisms that should be in place to track system performance and availability in real-time. Best pr

In [15]:

def extract_issues(text, id):
    pattern = r"Issue Type: (.*?)\nIssue Title: (.*?)\nExplanation: (.*?)\nSuggested Solution: (.*?)(?=\n---|$)"
    matches = re.findall(pattern, text, re.DOTALL)
    
    data = []
    for i, match in enumerate(matches):
        issue_type, issue_title, explanation, solution = [item.strip() for item in match]
        data.append({
            'id': id,
            'type': issue_type,
            'title': issue_title,
            'explanation': explanation,
            'solution': solution
        })
    
    return data

def extract_summary_numbers(text, id):
    
    pattern = r"Total incompleteness issues found: (\d+)\n- Total violations found: (\d+)"
    match = re.search(pattern, text)
    if match:
        incompleteness_issues = int(match.group(1))
        violations_issues = int(match.group(2))
        return {
            'id': id,
            'incompleteness': incompleteness_issues,
            'violation': violations_issues
        }
    return None


def append_to_csv(summary_data, csv_filename):
    df = pd.DataFrame([summary_data])  # Put the dictionary in a list to make it a row
    os.makedirs(os.path.dirname(csv_filename), exist_ok=True)

    if not os.path.isfile(csv_filename):
        df.to_csv(csv_filename, index=False, header=['id', 'incompleteness', 'violation'])
    else:
        df.to_csv(csv_filename, mode='a', header=False, index=False)

In [16]:
MODEL = "GPT-CE"
folder_path = f'output/query_results/{MODEL}/prompt_1'
output_folder_path = f'output/evaluations/prompt_1/{MODEL}'
csv_filename = f"output/evaluations/prompt_1/{MODEL}.csv"


for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            req_content = file.read()
            summary_data = extract_summary_numbers(req_content, id=filename)
            if summary_data:
                append_to_csv(summary_data, csv_filename)

# Prompt 2

In [39]:
MODEL = "GPT-OP"
folder_path = f'output/search_results/{MODEL}'
output_folder_path = f'output/query_results/{MODEL}/prompt_2'

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            req_type = "Non-Functional requirement" if "nfr" in filename else "Functional requirement"
            reference_text = file.read()

            # Load the requirement from file
            with open(f"documents/requirements/{filename}", 'r') as req_file:
                requirement = req_file.read()

            # Step 1: Run Requirement Analysis (Prompt 1)
            user_prompt_step1 = prompt_template.prompt_query_analysis(requirement, req_type)
            system_prompt = f"You are a software requirement analyst trained on standards, articles, and best practices contained in the following reference material: {reference_text}."
            result_step1 = call_LLM(user_prompt_step1, system_prompt)
            components_and_processes = result_step1.choices[0].message.content
            print(f"Step 1 Result (Components and Processes):\n{components_and_processes}")
            break
            # Step 2: Run Reference Material Review and Comparison (Prompt 2)
            user_prompt_step2 = prompt_template.prompt_query_review_and_comparison(requirement, components_and_processes, reference_text)
            result_step2 = call_LLM(user_prompt_step2, system_prompt)
            deviations_or_omissions = result_step2.choices[0].message.content
            print(f"Step 2 Result (Deviations or incompletness):\n{deviations_or_omissions}")

            # Step 3: Run Critical Deficiency Explanation and Final Assessment (Prompt 3)
            user_prompt_step3 = prompt_template.prompt_query_critical_deficiencies_and_assessment(deviations_or_omissions, reference_text)
            result_step3 = call_LLM(user_prompt_step3, system_prompt)
            final_analysis = result_step3.choices[0].message.content
            print(f"Step 3 Result (Final Analysis):\n{final_analysis}")

            # Save the final result to output folder
            output_file_path = os.path.join(output_folder_path, filename)
            with open(output_file_path, 'w') as output_file:
                output_file.write(final_analysis)
                print(f"Written final analysis to {output_file_path}")


AttributeError: module 'prompt_template' has no attribute 'prompt_query_analysis'

In [17]:
MODEL = "GPT-OP"
folder_path = f'output/search_results/{MODEL}'
output_folder_path = f'output/query_results/{MODEL}/prompt_2'


for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            req_type = ""
            if "nfr" in filename:
                req_type = "Non-Functional requirement"
            else:
                req_type = "Functional requirement"
            reference_text = file.read()
            with open("documents/requirements/" + filename, 'r') as req_file:
                requirement = req_file.read()
                user_prompt = prompt_template.prompt_query_graph_II(requirement, req_type)
                system_prompt = f"You are a software requirement analyst trained on standards, articles, and best practices contained in the following reference material: {reference_text}."
                result = call_LLM(user_prompt, system_prompt)
                text_result = result.choices[0].message.content
                print(text_result)

                output_file_path = os.path.join(output_folder_path, filename)
                with open(output_file_path, 'w') as output_file:
                    output_file.write(text_result)
                    print(f"Written content to {output_file_path}")

            


SUMMARY:
- Total incompleteness issues found: 2
- Total violations found: 0

DETAILED ANALYSIS:

Deficiencies Found:

1. 
   - Deficiency Type: Incompleteness
   - Deficiency Title: Lack of Load Testing Criteria
   - Deficiency: The requirement does not specify the need for load testing criteria to validate the system's performance under the expected load of 4 million simultaneous users. Load testing is essential to ensure that the system can handle the specified performance metrics, particularly during peak trading hours.
   - Suggested solution: It is crucial to define load testing scenarios that simulate the expected user load and transaction volume. This should include specific metrics for success, such as response times and throughput rates during testing. According to the IEEE 829 standard for software testing documentation, it is essential to include load testing as part of the performance testing strategy to ensure that the system meets its performance requirements under realis

In [18]:
MODEL = "BM25"
folder_path = f'output/query_results/{MODEL}/prompt_2'
output_folder_path = f'output/evaluations/prompt_2/{MODEL}'
csv_filename = f"output/evaluations/prompt_2/{MODEL}.csv"


for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            req_content = file.read()
            summary_data = extract_summary_numbers(req_content, id=filename)
            if summary_data:
                append_to_csv(summary_data, csv_filename)

# Prompt 3

In [34]:
import re

def extract_components_from_result(result_text):
    components = {}
    current_component = None
    current_interpretations = []

    for line in result_text.splitlines():
        line = line.strip()

        component_match = re.match(r'^(###\s*Component\s*\d+:\s*|Component\s*\d+:\s*)(.+)', line)  
        if component_match:
            if current_component and current_interpretations:
                components[current_component] = current_interpretations

            current_component = component_match.group(2).strip()  
            current_interpretations = [] 

        interpretation_match = re.match(r'^\d+\.\s*(.*)', line)
        if interpretation_match:
            current_interpretations.append(interpretation_match.group(1).strip())

    if current_component and current_interpretations:
        components[current_component] = current_interpretations

    return components

def extract_votes_from_exploration(exploration_text):
    votes = []

    for line in exploration_text.splitlines():
        line = line.strip()

        vote_match = re.match(r'^-.*Preliminary Vote.*:\s*(\d+)', line)
        if vote_match:
            vote = int(vote_match.group(1))  
            votes.append(vote)

    return votes



In [5]:
MODEL = "GPT-OP"
folder_path = f'output/search_results/GPT-OP'
output_folder_path = f'output/query_results/{MODEL}/prompt_3'

threshold = 3  

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            req_type = "Non-Functional requirement" if "nfr" in filename else "Functional requirement"
            reference_text = file.read()
            
            with open(f"documents/requirements/{filename}", 'r') as req_file:
                requirement = req_file.read()

            # Initial Breakdown (Step 1)
            initial_breakdown_prompt = prompt_template.prompt_query_initial_breakdown(requirement, req_type)
            system_prompt = f"You are a software requirement analyst trained on standards, articles, and best practices contained in the following reference material: {reference_text}."
            initial_breakdown_result = call_LLM(initial_breakdown_prompt, system_prompt)
            initial_breakdown_text = initial_breakdown_result.choices[0].message.content
            #print(initial_breakdown_text, "\n!!!!!!!!!!")
            components = extract_components_from_result(initial_breakdown_text)  

            # Exploration and Voting (Step 2) with pruning
            exploration_results = []
            for component, interpretations in components.items():
                exploration_prompt = prompt_template.prompt_query_exploration_voting(requirement, component, interpretations)
                exploration_result = call_LLM(exploration_prompt, system_prompt)
                exploration_text = exploration_result.choices[0].message.content
                print(exploration_text)
                
                votes = extract_votes_from_exploration(exploration_text)  
                
                kept_interpretations = []
                for interpretation, vote in zip(interpretations, votes):
                    if vote >= threshold:
                        kept_interpretations.append(interpretation)
                
                if kept_interpretations: 
                    exploration_results.append((component, kept_interpretations))
            
            # Evaluation and Selection (Step 3)
            selected_interpretations = {}
            for component, interpretations in exploration_results:
                evaluation_prompt = prompt_template.prompt_query_evaluation_selection(requirement, component, interpretations)
                evaluation_result = call_LLM(evaluation_prompt, system_prompt)
                selected_interpretations[component] = evaluation_result.choices[0].message.content

            # Deep Dive and Refined Voting (Step 4)
            deep_dive_results = {}
            for component, selected_interps in selected_interpretations.items():
                deep_dive_prompt = prompt_template.prompt_query_deep_dive_voting(requirement, component, selected_interps)
                deep_dive_result = call_LLM(deep_dive_prompt, system_prompt)
                deep_dive_results[component] = deep_dive_result.choices[0].message.content

            # Synthesis and Final Voting (Step 5)
            synthesis_results = []
            for component, deep_dive_result in deep_dive_results.items():
                synthesis_prompt = prompt_template.prompt_query_synthesis_final_voting(requirement, component, deep_dive_result)
                synthesis_result = call_LLM(synthesis_prompt, system_prompt)
                synthesis_results.append(synthesis_result.choices[0].message.content)

            # Combine final results
            final_result = "\n".join(synthesis_results)

            output_file_path = os.path.join(output_folder_path, filename)
            with open(output_file_path, 'w') as output_file:
                output_file.write(final_result)
                print(f"Written content to {output_file_path}")


### Component 1: Response Time

- **Interpretations:**
  1. **User Experience Focus:** The response time requirement ensures that users experience minimal delay when executing trades, which is crucial for maintaining satisfaction and trust in a high-stakes trading environment.
  2. **System Load Management:** The requirement implies that the system must efficiently manage resources and optimize processing to handle the peak load of 4 million users simultaneously without exceeding the 500-millisecond threshold.
  3. **Variable Conditions:** The response time can vary based on external factors such as stock exchange rate changes, suggesting that the system must be adaptable and resilient to fluctuations in external data sources.

### Component 2: Availability and Uptime

- **Interpretations:**
  1. **Operational Reliability:** The 99% uptime requirement indicates a high level of reliability, ensuring that the system is consistently operational during the critical trading window, minimizi

In [27]:
import os

def remove_asterisks_from_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):  
            file_path = os.path.join(folder_path, filename)

            with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                file_content = file.read()

            updated_content = file_content.replace('*', '')

            with open(file_path, 'w', encoding='utf-8', errors='ignore') as file:
                file.write(updated_content)

    print("Asterisks removed from all text files in the folder.")


def count_deficiencies(text, id):
    incompleteness_count = len(re.findall(r"Deficiencies type: Incompleteness", text))
    violation_count = len(re.findall(r"Deficiencies type: Violation", text))
    
    return {
        'id': id,
        'incompleteness_count': incompleteness_count,
        'violation_count': violation_count
    }


In [30]:
MODEL = "GPT-OP"
folder_path = f'output/query_results/{MODEL}/prompt_3'
output_folder_path = f'output/evaluations/prompt_3/{MODEL}'
csv_filename = f"output/evaluations/prompt_3/{MODEL}.csv"

remove_asterisks_from_files(folder_path)

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            req_content = file.read()
            count_data = count_deficiencies(req_content, id=filename)
            append_to_csv(count_data, csv_filename)

Asterisks removed from all text files in the folder.
